In [ ]:
from my_packages import *
from appgeopy import *

#### GPS data

In [ ]:
GPS_csvfolder = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2024_HsuYaru\2__Processed\A__Fulltime"

In [ ]:
GPS_gdf = gpd.read_file(r"D:\003_PRESENTATIONS\20_TSX_Sentinel_Report\geofiles\GPS_tsx_angle.shp")
GPS_gdf.crs

#### TSX data

In [ ]:
TSX_PS_gdf = pd.read_pickle(r"D:\SharedFolder\ISCE_Mintpy\TSX_Run014_PS002\TSX_ref2DNAN_WGS84.pkl", compression="zip")
TSX_PS_gdf.head(5)

In [ ]:
nearby_psc_df = pd.read_pickle("TSX_PSC_nearbyGPS_500m.pkl", compression='zip')
nearby_psc_df.head(5)

select stations and available CSV files

In [ ]:
available_stations = nearby_psc_df["STATION"].unique()
GPS_csvfiles = {}
for station in available_stations:
    temp_fpath = os.path.join(GPS_csvfolder, station+".csv")
    # print(temp_fpath)
    if os.path.isfile(temp_fpath):
        GPS_csvfiles[station] = temp_fpath

In [ ]:
GPS_csvfiles

In [ ]:
select_station = "HUWE"

GPSstation_csvfile = os.path.join(GPS_csvfolder, f"{select_station}.csv")
GPSstation_df = pd.read_csv(GPSstation_csvfile, parse_dates=[0], index_col=[0])

azim, inc = GPS_gdf.query("STATION==@select_station")[["AZIMUTH", "Incidence"]].values.flatten()

dN, dE, dU = (GPSstation_df.iloc[:, i] for i in range(GPSstation_df.shape[1]))
LOS_velocity = ts_disp.get_LOS_disp(dN, dE, dU, incidence_angle=inc, heading_angle=azim)

In [ ]:
pointkey_arr = nearby_psc_df.query("STATION==@select_station")["PointKey"]
TSX_PS_byPointKey = TSX_PS_gdf.query("PointKey in @pointkey_arr")

# columns starting with N
measurement_columns = [col for col in TSX_PS_byPointKey.columns if col.startswith("N")]

# extract table with measurement data
TSX_PS_byPointKey = TSX_PS_byPointKey.loc[:, measurement_columns]

# reference to the first day
TSX_PS_byPointKey = TSX_PS_byPointKey.subtract(TSX_PS_byPointKey.iloc[:, 0], axis=0)

# transpose the data table
TSX_PS_byPointKey = TSX_PS_byPointKey.T

# adjusting the data table information
TSX_PS_byPointKey.columns = pointkey_arr
TSX_PS_byPointKey.index = pd.to_datetime([ele[1:] for ele in TSX_PS_byPointKey.index])

In [ ]:
# InSAR data not matching the time with GPS data
# need to extract the time with first valid measurement
first_valid_idx_TSX = TSX_PS_byPointKey.first_valid_index()

# extract the GPS measurement starting from
# the first InSAR valid time index
LOS_vel_byTSX_idx = LOS_velocity.loc[first_valid_idx_TSX:]

# some GPS contains missing values,
# need to get the first valid index of GPS data
# to align 2 time-series
first_valid_idx_LOS_arr = LOS_vel_byTSX_idx.first_valid_index()

# extract the LOS array from the first valid index
valid_LOS_vel_arr = LOS_velocity.loc[first_valid_idx_LOS_arr:]
# reference measurements to the first valid measurement
valid_LOS_vel_arr_ref2first = valid_LOS_vel_arr.subtract(valid_LOS_vel_arr.iloc[0])
valid_LOS_vel_arr_ref2first = valid_LOS_vel_arr_ref2first*1000 # convert to milimeter

# align the time with GPS array
TSX_PS_byPointKey_LOS_align = TSX_PS_byPointKey.loc[first_valid_idx_LOS_arr:, :]
# reference measurements to the first valid measurement
TSX_PS_byPointKey_LOS_align = TSX_PS_byPointKey_LOS_align.subtract(TSX_PS_byPointKey_LOS_align.iloc[0, :])

#### plot GPS and TSX PS measurements

In [ ]:
fig = plt.figure(figsize=(12,4))

ax = fig.add_subplot(111)
ax.plot(TSX_PS_byPointKey_LOS_align, color='lightgrey', ls=' ', marker='o', alpha=0.3)
ax.plot(TSX_PS_byPointKey_LOS_align.mean(axis=1), color='blue', ls='--', marker='s', ms=10, alpha=0.3)
ax.plot(valid_LOS_vel_arr_ref2first, color='black', lw=2)
plt.show()